# Serving for Data Analytics

In the first part of the lab, you ingested data from a source database and transformed it using AWS Glue. In this notebook, you will serve the transformed data for an analytics example. You will perform data retrieval with Amazon Athena using simple SQL queries and then use the query output to build an interactive dashboard that explores sales data by country and product line.

Import all of the required packages.

In [1]:
import pandas as pd
import awswrangler as wr
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import warnings

After you have executed the AWS Glue job, a new database named `de-c1w2-analytics-db` was created and stored in the S3 instance. This database contains four tables with the following schema:

![image alt ><](./images/schema_after_ETL.png)

Each row in the `fact_orders` table corresponds to a sale order and contains related measurements such as quantity ordered and price. The dimension tables provide more context and details for each sale order such as customers' information, customers' locations, and order details. Your data is now ready to be served for analytics. To query this data from S3, you will use Amazon Athena. Let's check the data stored in the `dim_products` table:

In [2]:
GLUE_DATABASE = "de-c1w2-analytics-db"

products_df = wr.athena.read_sql_query(
    """
    SELECT * FROM dim_products
    """,
    database=GLUE_DATABASE,
)
    
products_df.head()

/home/coder/miniconda/lib/python3.12/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by promote_options='default'.
  return cls.dispatch_func(func)(*args, **kw)


,productcode,productname,productline,productscale,productvendor,productdescription,productlinedescription
0,S32_1374,1997 BMW F650 ST,Motorcycles,1:32,Exoto Designs,Features official die-struck logos and baked e...,Our motorcycles are state of the art replicas ...
1,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",Our motorcycles are state of the art replicas ...
2,S24_1578,1997 BMW R 1100 S,Motorcycles,1:24,Autoart Studio Design,Detailed scale replica with working suspension...,Our motorcycles are state of the art replicas ...
3,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",Our motorcycles are state of the art replicas ...
4,S32_4485,1974 Ducati 350 Mk3 Desmo,Motorcycles,1:32,Second Gear Diecast,This model features two-tone paint with chrome...,Our motorcycles are state of the art replicas ...


You can get some data insights by aggregating some sale measurements from the fact table. In the following cell, you will compute the total sales amount spent by each country and display the top 10 records:

In [3]:
# Retrieve total sales per country by joining the 'fact_orders' table with 'dim_locations' on postal code.
# The result is grouped by the 'country' column, and the total sales for each country is calculated as the sum of order amounts.
# The query is executed using Athena through the 'wr.athena.read_sql_query' method, and the top 10 countries with the highest total sales are displayed.
product_sales_by_country_df = wr.athena.read_sql_query(
    """
    SELECT
        dim_locations.country,
        SUM(fact_orders.orderAmount) AS total_sales
    FROM
        fact_orders
    JOIN
        dim_locations ON fact_orders.postalCode = dim_locations.postalCode
    GROUP BY 1
    """,
    database=GLUE_DATABASE,
)
    
product_sales_by_country_df.sort_values("total_sales", ascending=False).head(10)

/home/coder/miniconda/lib/python3.12/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by promote_options='default'.
  return cls.dispatch_func(func)(*args, **kw)


,country,total_sales
9,USA,13057249.77
8,Spain,1099389.09
4,France,1007374.02
0,Australia,562582.59
12,UK,436947.44
2,Italy,335258.49
1,Finland,295149.35
13,Singapore,263997.78
14,Denmark,218994.92
11,Canada,205911.86


Now you will combine data from three tables: `fact_orders`, `dim_products`, and `dim_locations`. The query will select the order date, product line, product name, country, and total sales amount, grouping the results by order date, product line, product name, and country:

In [4]:
# Retrieve detailed sales information by joining 'fact_orders' with 'dim_products' and 'dim_locations'.
# The query selects order date, product line, product name, country, and calculates the total sales (sum of order amounts).
# The result is grouped by order date, product line, product name, and country.
# The query is executed using Athena through the 'wr.athena.read_sql_query' method, and the resulting DataFrame is displayed with the first few rows using 'head()'.
product_sales_df = wr.athena.read_sql_query(
    """
    SELECT
        fact_orders.orderDate,
        dim_products.productLine,
        dim_products.productName,
        dim_locations.country,
        SUM(fact_orders.orderAmount) AS total_sales
    FROM
        fact_orders
    JOIN
        dim_products ON fact_orders.productCode = dim_products.productCode
    JOIN
        dim_locations ON fact_orders.postalCode = dim_locations.postalCode
    GROUP BY 1, 2, 3, 4
    """,
    database=GLUE_DATABASE,
)
    
product_sales_df.head()

/home/coder/miniconda/lib/python3.12/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by promote_options='default'.
  return cls.dispatch_func(func)(*args, **kw)


,orderdate,productline,productname,country,total_sales
0,2004-06-15,Trucks and Buses,1996 Peterbilt 379 Stake Bed with Outrigger,Japan,1254.00
1,2005-01-26,Classic Cars,1969 Ford Falcon,Japan,4765.10
2,2004-11-23,Motorcycles,2003 Harley-Davidson Eagle Drag Bike,Japan,6948.63
3,2004-06-15,Trucks and Buses,1940 Ford Pickup Truck,Japan,4639.97
4,2005-01-26,Vintage Cars,1904 Buick Runabout,Japan,3238.59


You will now use the query results to build an interactive dashboard using dropdown widgets, where you will be able to select a country and product line, filter the results based on a particular period of sales, showing the top N popular products at the end:

In [5]:
product_sales_df['orderdate'] = pd.to_datetime(product_sales_df['orderdate'])

# Ignore warnings for cleaner output
warnings.filterwarnings('ignore')

# Country Dropdown Widget
country_widget = widgets.Dropdown(
    options=["ALL"] + sorted(list(product_sales_df.country.unique())),
    value="ALL",
    description="Country",
)

# Product Line Dropdown Widget
productline_widget = widgets.Dropdown(
    options=["ALL"] + sorted(list(product_sales_df.productline.unique())),
    value="ALL",
    description="Product Line",
)

# Interactive Widgets for Date Range, Country, Product Line, and Top N
@widgets.interact(
    start_date=widgets.DatePicker(value=product_sales_df.orderdate.min(), description="Start Date"),
    end_date=widgets.DatePicker(value=product_sales_df.orderdate.max(), description="End Date"),
    country=country_widget,
    productline=productline_widget,
    top_n=widgets.IntSlider(value=5, min=1, max=10, step=1, description="Top N"),
)

# Function to Plot Top N Sales
def plot_top_n_sales(start_date, end_date, country, productline, top_n):
    
    # Filter data based on selected date range
    filtered_df = product_sales_df[
        (product_sales_df.orderdate >= pd.Timestamp(start_date))
        & (product_sales_df.orderdate <= pd.Timestamp(end_date))
    ]
    filtered_df = filtered_df.drop('orderdate', axis=1)
    filtered_df = filtered_df.groupby(['productline', 'productname', 'country']).sum().reset_index()

    # Build title string based on selected filters
    title_str = f"Top {top_n} Popular "
    
    if productline != "ALL":
        filtered_df = filtered_df[filtered_df.productline == productline]
        title_str += productline
    else: 
        filtered_df = filtered_df.groupby(['productname', 'country']).sum().reset_index()
        title_str += "Products"
        
    if country != "ALL":
        filtered_df = filtered_df[filtered_df.country == country]
        title_str += " in " + country
    else:
        filtered_df = filtered_df.groupby(['productname']).sum().reset_index()

    # Plotting the bar chart
    if not (filtered_df.empty):
        try:
            ax = sns.barplot(
                x="total_sales",
                y="productname",
                data=filtered_df.sort_values("total_sales", ascending=False).head(top_n)
            )
        

            ax.set(
                xlabel="Total Sales",
                ylabel="Product Name",
                title=title_str
            )
        except:
            print("error")
    else:
        print(f"There were no sales of {productline} to {country} during that period")

interactive(children=(DatePicker(value=Timestamp('2003-01-06 00:00:00'), description='Start Date', step=1), Da…

Fantastic! You can now observe how effortlessly the data can be accessed after having transformed it into a form that is more appropriate for analytics.